In [1]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev

Get:1 http://ports.ubuntu.com/ubuntu-ports jammy InRelease [270 kB]
Get:2 http://ports.ubuntu.com/ubuntu-ports jammy-updates InRelease [128 kB]
Get:3 http://ports.ubuntu.com/ubuntu-ports jammy-backports InRelease [127 kB]
Get:4 http://ports.ubuntu.com/ubuntu-ports jammy-security InRelease [129 kB]
Get:5 http://ports.ubuntu.com/ubuntu-ports jammy/main arm64 Packages [1,758 kB]
Get:6 http://ports.ubuntu.com/ubuntu-ports jammy/restricted arm64 Packages [24.2 kB]
Get:7 http://ports.ubuntu.com/ubuntu-ports jammy/multiverse arm64 Packages [224 kB]
Get:8 http://ports.ubuntu.com/ubuntu-ports jammy/universe arm64 Packages [17.2 MB]
Get:9 http://ports.ubuntu.com/ubuntu-ports jammy-updates/universe arm64 Packages [1,375 kB]
Get:10 http://ports.ubuntu.com/ubuntu-ports jammy-updates/main arm64 Packages [2,176 kB]
Get:11 http://ports.ubuntu.com/ubuntu-ports jammy-updates/multiverse arm64 Packages [29.5 kB]
Get:12 http://ports.ubuntu.com/ubuntu-ports jammy-updates/restricted arm64 Packages [2,352 kB]

In [5]:
#!pip install tensorflow-probability==0.8.0
!export HDF5_DIR=/usr/local && pip install --no-binary=h5py h5py &&pip install tf-agents tf-keras --upgrade --force-reinstall

  Using cached tf_agents-0.19.0-py3-none-any.whl.metadata (12 kB)
  Using cached tf_keras-2.17.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached cloudpickle-3.0.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached gin_config-0.5.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached gym-0.23.0-py3-none-any.whl
  Using cached numpy-2.1.1-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (62 kB)
  Using cached pillow-10.4.0-cp311-cp311-manylinux_2_28_aarch64.whl.metadata (9.2 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached protobuf-5.28.0-cp38-abi3-manylinux2014_aarch64.whl.metadata (592 bytes)
  Using cached wrapt-1.16.0-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (6.6 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached pygame-2.1.3-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (9.

In [55]:
!pip install tensorflow --upgrade
!pip install scikit-image --upgrade
!pip install tf-agents
!pip install pyglet xvfbwrapper
!pip install tf-keras
!pip install keras-tuner
!pip install pandas pandas-ta numpy

  Using cached imageio-2.35.1-py3-none-any.whl.metadata (4.9 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached numpy-2.1.1-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (62 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached dm_tree-0.1.8-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (1.9 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached h5py-3.11.0.tar.gz (406 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached libclang-18.1.1-py2.py3-none-manylinux2014_aarch64.whl.metadata (5.2 kB)
  Using cached ml_dtypes-0.4.0-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (20 kB)
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
  Using cached protobuf-4.25.4-cp37-abi3-manylinux2014_aarch64.whl.metadata (541

In [ ]:
from packaging import version

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

In [57]:
import tensorflow as tf
from tf_agents.networks.q_network import QNetwork

class DuelingQNetwork(QNetwork):
    def __init__(self, input_tensor_spec, action_spec, fc_layer_params=(100,), name='DuelingQNetwork'):
        super(DuelingQNetwork, self).__init__(input_tensor_spec, action_spec, fc_layer_params, name)
        
        # Number of actions
        num_actions = action_spec.maximum - action_spec.minimum + 1
        
        # Define the shared layers
        self.shared_layers = tf.keras.Sequential([
            tf.keras.layers.Dense(fc_layer_params[0], activation='relu'),
            tf.keras.layers.Dense(fc_layer_params[0], activation='relu'),
        ])
        
        # Define the value stream
        self.value_stream = tf.keras.Sequential([
            tf.keras.layers.Dense(fc_layer_params[0], activation='relu'),
            tf.keras.layers.Dense(1),  # Outputs a single state value V(s)
        ])
        
        # Define the advantage stream
        self.advantage_stream = tf.keras.Sequential([
            tf.keras.layers.Dense(fc_layer_params[0], activation='relu'),
            tf.keras.layers.Dense(num_actions),  # Outputs advantage A(s, a) for each action
        ])

    def call(self, observation, step_type=None, network_state=(), training=False):
        """Returns Q-values for each action."""
        # Pass the input through the shared layers
        x = self.shared_layers(observation)
        
        # Compute state value and advantages
        value = self.value_stream(x)
        advantage = self.advantage_stream(x)
        
        # Combine value and advantage to get Q-values
        q_values = value + (advantage - tf.reduce_mean(advantage, axis=-1, keepdims=True))
        
        return q_values, network_state


AttributeError: module 'keras._tf_keras.keras' has no attribute '__internal__'